In [ ]:
%%time

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from polygraphy.backend.trt import util as trt_util
import torch
from collections import OrderedDict

from cuda import cudart
from copy import deepcopy
import tensorrt as trt

import pycuda.driver as cuda
import pycuda.autoinit




In [ ]:
device = "cuda"

In [ ]:

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)


def show_mask(mask, ax):
    color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


class ImageTensor:
    def __init__(self, image):
        self.image = image
        self.orig_width, self.orig_height = image.size
        self.resized_width, self.resized_height = None, None
        self.pad_width, self.pad_height = None, None

    def size(self):
        return self.image.size

    def apply_coords(self, coords: np.ndarray) -> np.ndarray:
        """
        Expects a numpy array of length 2 in the final dimension
        """
        old_h, old_w = self.orig_height, self.orig_width
        new_h, new_w = self.resized_height, self.resized_width
        coords = deepcopy(coords).astype(float)
        coords[..., 0] = coords[..., 0] * (new_w / old_w)
        coords[..., 1] = coords[..., 1] * (new_h / old_h)
        return coords


class ImagePreprocessor:
    def __init__(self, long_side_max=1024, mean=None, std=None, image_format="RGB", pad_to_square=True):
        self.long_side_max = long_side_max
        self.mean = mean
        self.std = std
        self.image_format = image_format
        self.pad_to_square = pad_to_square
        if self.mean is None:
            self.mean = np.array([123.675, 116.28, 103.53])
        if self.std is None:
            self.std = np.array([58.395, 57.12, 57.375])


    def resize_image_to_long_side(self, img: ImageTensor):
        if self.long_side_max is None:
            return img
        orig_width, orig_height = img.image.size
        if orig_width > orig_height:
            img.resized_width = self.long_side_max
            img.resized_height = int(self.long_side_max / orig_width * orig_height)
        else:
            img.resized_height = self.long_side_max
            img.resized_width = int(self.long_side_max / orig_height * orig_width)

        img.image = img.image.resize((img.resized_width, img.resized_height), Image.Resampling.BILINEAR)
        return img

    def make_image_rgb(self, image):
        if image.image.mode == "RGB":
            return image
        else:
            image.image = image.image.convert("RGB")
            return image

    def pad_image_to_square(self, image):
        if isinstance(image, ImageTensor):
            image.image = self.pad_image_to_square(image.image)
            return image
        else:
            h, w = image.shape[2:]
            max_dim = max(h, w)
            pad_h = max_dim - h
            pad_w = max_dim - w
            image = np.pad(image, ((0,0), (0,0), (0,pad_h), (0,pad_w)), mode="constant", constant_values=0)
            return image

    def normalize_image(self, image):
        if isinstance(image, ImageTensor):
            image.image = self.normalize_image(image.image)
            return image
        else:
            image = (image - self.mean) / self.std
            return image

    def to_tensor(self, image):
        if isinstance(image, ImageTensor):
            image.image = self.to_tensor(image.image)
            return image
        else:
            image = image.transpose(2,0,1)[None,:,:,:].astype(np.float32)
            return image

    def from_image_to_input(self, image):
        image = self.make_image_rgb(image)
        image = self.resize_image_to_long_side(image)
        image = self.normalize_image(image)
        image = self.to_tensor(image)
        # pad to square
        if self.pad_to_square:
            image = self.pad_image_to_square(image)
        return image


In [ ]:
import os
print(os.getcwd())

In [ ]:
f = open("engine_fp32/decoder.engine", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))

decoder_engine = runtime.deserialize_cuda_engine(f.read())
decoder_context = decoder_engine.create_execution_context()

In [ ]:
f = open("engine_fp32/encoder.engine", "rb")
encoder_engine = runtime.deserialize_cuda_engine(f.read())
encoder_context = encoder_engine.create_execution_context()

In [ ]:
# open sample file
image_path = R"downloads/sample.png"
image = Image.open(image_path)

# image collections for plotting and testing
image_original = ImageTensor(image)
image_object = ImageTensor(image)

# image processor object
image_preprocessor = ImagePreprocessor()

image_rgb = image_preprocessor.make_image_rgb(image_object)
numpy_image = np.array(image_rgb.image)
input_image = image_preprocessor.from_image_to_input(image_object)

In [ ]:
input_image.resized_height, input_image.resized_width, input_image.orig_height, input_image.orig_width

In [ ]:
plt.imshow(input_image.image[0,0,...])

In [ ]:
input_tensor = input_image.image

In [ ]:
input_tensor.shape, input_tensor.dtype

In [ ]:
import numpy as np
BATCH_SIZE = 1
USE_FP16 = False
target_dtype = np.float16 if USE_FP16 else np.float32
# need to set input and output precisions to FP16 to fully enable it
output = np.empty([BATCH_SIZE, 256, 64, 64], dtype = target_dtype) 

# allocate device memory for encoder
d_input = cuda.mem_alloc(1 * input_tensor.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)
bindings = [int(d_input), int(d_output)]



In [ ]:
stream = cuda.Stream()

In [ ]:
def predict_encoder(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    encoder_context.execute_async_v2(bindings, stream.handle, None)
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()
    
    return output

In [ ]:
print("Warming up...")
pred = predict_encoder(input_tensor)
print("Done warming up!")

In [ ]:
%%timeit
pred = predict_encoder(input_tensor)

In [ ]:
pred.dtype, type(pred), pred.shape

In [ ]:
# plot the input embedding
plt.imshow(pred[0,0,:42,:])

# Decoder

In [ ]:
# image_size = np.array([image_rgb.orig_width, image_rgb.orig_height ], dtype=np.float32)
# #plotting a query point on the image
# input_point = np.array([[300, 300]])
# # label 1 means foreground, 0 means background
# input_label = np.array([1])
# # format of the box is [x0, y0, x1, y1], where o is the top left corner and 1 is the bottom right corner
# input_box = np.array([400, 250, 2500, 1500])
# box_labels = np.array([2, 3])

# onnx_box_coords = input_box.reshape(2, 2)
# onnx_box_labels = np.array([2,3])

# onnx_coord = np.concatenate([input_point, onnx_box_coords], axis=0)[None, :, :]
# onnx_label = np.concatenate([input_label, onnx_box_labels], axis=0)[None, :].astype(np.float32)
# onnx_coord = input_image.apply_coords(onnx_coord).astype(np.float32)
# # onnx_coord = predictor.transform.apply_coords(onnx_coord, image_size).astype(np.float32)

# onnx_mask_input = np.zeros((1, 1, 256, 256), dtype=np.float32)
# onnx_has_mask_input = np.zeros(1, dtype=np.float32)

# decoder_params = {
#     "image_embeddings": pred,
#     "point_coords": onnx_coord,
#     "point_labels": onnx_label,
#     "mask_input": onnx_mask_input,
#     "has_mask_input": onnx_has_mask_input,
# }

In [ ]:
def get_shape_dict(batch, points):
    embed_dim = 256
    embed_size = (64, 64)
    mask_input_size = [4 * x for x in embed_size]

    return {
        "image_embeddings": (1, embed_dim, *embed_size),
        "point_coords": (batch, points, 2),
        "point_labels": (batch, points),
        "mask_input": (1, 1, *mask_input_size),
        "has_mask_input": (1, ),
        "iou_predictions": (batch, 4), # up to 4 masks per point
        "low_res_masks": (batch, 4, *mask_input_size), # up to 4 masks per point
    }

In [ ]:

def get_param_dict(embeddings, image_embedding_size, device):
    input_point = torch.as_tensor(np.array([[300,300]]), device=device, dtype=torch.float32)
    input_label = torch.as_tensor(np.array([1]), device=device, dtype=torch.float32)
    zero_point = torch.zeros(1, 2, device=device, dtype=torch.float32)
    negative_label = torch.as_tensor(np.array([-1]), device=device, dtype=torch.float32)
    coord = torch.cat((input_point, zero_point), dim=0)[None, :, :]
    label = torch.cat((input_label, negative_label), dim=0)[None, :]
    dtype = torch.float32

    embeddings = torch.as_tensor(embeddings, dtype=dtype)
    coord = input_point.to(dtype=dtype)
    label = input_label.to(dtype=dtype)

    mask_input = torch.zeros(1, 1, 4 * image_embedding_size[0], 
                                4 * image_embedding_size[1], device=device, dtype=dtype)
    has_mask_input = torch.zeros(1, dtype=dtype, device=device)


    params = {
        "image_embeddings": embeddings,
        "point_coords": coord,
        "point_labels": label,
        "mask_input": mask_input,
        "has_mask_input": has_mask_input,
    }
    return params

In [ ]:
tensors = OrderedDict()
def _allocate_buffers(shape_dict, engine, context):
    for idx in range(trt_util.get_bindings_per_profile(engine)):
        
        binding = engine[idx]
        if shape_dict and binding in shape_dict:
            shape = shape_dict[binding]
        else:
            shape = engine.get_binding_shape(binding)
        dtype = trt.nptype(engine.get_tensor_dtype(binding))
        if engine.get_tensor_mode(binding) == trt.TensorIOMode.INPUT:
            context.set_input_shape( binding, shape)
        tensor = torch.empty(tuple(shape), dtype=torch.float32).to(device=device)
        tensors[binding] = tensor

In [ ]:
decoder_dict = get_shape_dict(batch=1, points=2)
print(decoder_dict)
decoder_params = get_param_dict(embeddings=pred, image_embedding_size=(64, 64), device=device)
for key, value in decoder_params.items():
    print(key, value.shape)
for key, value in tensors.items():
    print(key, value.shape)

In [ ]:
_allocate_buffers(decoder_dict, decoder_engine, decoder_context)

In [ ]:
def CUASSERT(cuda_ret):
    err = cuda_ret[0]
    if err != cudart.cudaError_t.cudaSuccess:
         raise RuntimeError(f"CUDA ERROR: {err}, error code reference: https://nvidia.github.io/cuda-python/module/cudart.html#cuda.cudart.cudaError_t")
    if len(cuda_ret) > 1:
        return cuda_ret[1]
    return None

def infer_engine(stream, tensors, param_dict, context, use_cuda_graph=True, cuda_graph_instance=None):
    for name, buf in param_dict.items():
        tensors[name].copy_(buf)

    for name, tensor in tensors.items():
        context.set_tensor_address(name, tensor.data_ptr())

    if use_cuda_graph:
        if cuda_graph_instance is not None:
            CUASSERT(cudart.cudaGraphLaunch(cuda_graph_instance, stream.handle))
            CUASSERT(cudart.cudaStreamSynchronize(stream.handle))
        else:
            # do inference before CUDA graph capture
            noerror = context.execute_async_v3(stream.handle)
            if not noerror:
                raise ValueError(f"ERROR: inference failed.")
            # capture cuda graph
            CUASSERT(cudart.cudaStreamBeginCapture(stream.handle, cudart.cudaStreamCaptureMode.cudaStreamCaptureModeGlobal))
            context.execute_async_v3(stream.handle)
            graph = CUASSERT(cudart.cudaStreamEndCapture(stream.handle))
            cuda_graph_instance = CUASSERT(cudart.cudaGraphInstantiate(graph, 0))
    else:
        noerror = context.execute_async_v3(stream.handle)
        if not noerror:
            raise ValueError(f"ERROR: inference failed.")
        
    stream.synchronize()


    return tensors


In [ ]:
outputs = infer_engine(stream, tensors, decoder_params, decoder_context, use_cuda_graph=True)

In [ ]:
for key, value in outputs.items():
    print(key, value.shape)

In [ ]:
masks = outputs['low_res_masks']
scores = outputs['iou_predictions']

In [ ]:
print(masks.dtype, masks.shape)

In [ ]:
import torch
from torchvision.transforms.functional import to_tensor
from torchvision.transforms.functional import to_pil_image

def upsample_image(image, new_size):
    if not isinstance(image, torch.Tensor):
        image_tensor = to_tensor(image)
    else:
        image_tensor = image
    
    image_tensor = image_tensor.unsqueeze(0).to(torch.device('cuda'))
    upsampled_tensor = torch.nn.functional.interpolate(image_tensor, size=new_size, mode='bilinear', align_corners=False)
    upsampled_tensor = upsampled_tensor.squeeze(0).to(torch.device('cpu')).numpy()
    
    return upsampled_tensor

def crop_image_to_prepadded_size(image, padding_size):
    width, height = image.size
    left = padding_size
    top = padding_size
    right = width - padding_size
    bottom = height - padding_size
    cropped_image = image.crop((left, top, right, bottom))
    return cropped_image


In [ ]:
sliced_masks = masks[:,3, :, :]
upsampled_mask = upsample_image(sliced_masks, (1024, 1024))
print(upsampled_mask.shape)

In [ ]:
# crop to prepadded size
padding_size = np.asarray(upsampled_mask.shape[-2:])- np.asarray(image_original.image.size)
cropped_mask = upsampled_mask[0, :-padding_size[0], :-padding_size[1]]
cropped_mask = cropped_mask > 0

In [ ]:
plt.imshow(cropped_mask)

In [ ]:
np.any(cropped_mask)

In [ ]:
def _mask_postprocessing(self, masks: torch.Tensor, orig_im_size) -> torch.Tensor:
    orig_im_size_torch = torch.tensor(orig_im_size, device=self.device)
    masks = F.interpolate(
        masks,
        size=(1024, 1024),
        mode="bilinear",
        align_corners=False,
    )
    
    masks = F.interpolate(masks, size=(h, w), mode="bilinear", align_corners=False)
    return masks

In [ ]:
upscaled_masks = self._mask_postprocessing(masks, orig_im_size)

upscaled_masks = upscaled_masks > self.sam.mask_threshold